In [1]:
# step 1: loading the necessary libraries and start the graph session
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
sess=tf.Session()

In [246]:
# step 2: load the data and transform  he images into 28*28 arrays
data_dir='temp'
mnist=read_data_sets(data_dir)
train_xdata=np.array([np.reshape(x, (28, 28)) for x in mnist.train.images])
test_xdata=np.array([np.reshape(x, (28, 28)) for x in mnist.test.images ])
train_labels=mnist.train.labels
test_labels=mnist.test.labels

Extracting temp\train-images-idx3-ubyte.gz
Extracting temp\train-labels-idx1-ubyte.gz
Extracting temp\t10k-images-idx3-ubyte.gz
Extracting temp\t10k-labels-idx1-ubyte.gz


In [247]:
print(train_xdata.shape)
print(train_labels.shape)

(55000, 28, 28)
(55000,)


In [270]:
# step:3 define the model hyper-parameters 
batch_size=100
learning_rate=0.05
evaluation_size=100
image_width=train_xdata[0].shape[0]
image_height=train_xdata[0].shape[1]
target_size= 1
num_channel=1
generations=500
eval_every=5
conv1_features=28
conv2_features=50
max_pool_size1=2
max_pool_size2=2
fully_connected_size1=100

In [271]:
#step: 4 decleare placeholder for the data.
x_input_shape=(batch_size, num_channel, image_width, image_height)
x_input=tf.placeholder(tf.float32, shape=x_input_shape)
y_target=tf.placeholder(tf.int32, shape=(batch_size, 1))
eval_input_shape=(evaluation_size, num_channel, image_width, image_height)
eval_input=tf.placeholder(tf.float32, shape=eval_input_shape)
eval_target=tf.placeholder(tf.int32, shape=(batch_size, 1))

In [272]:
# step 5: declare weight and bias Variables for CNN network
conv1_weight=tf.Variable(tf.truncated_normal([4, 4,  conv1_features, conv1_features], stddev=0.1, dtype=tf.float32))
conv1_bias=tf.Variable(tf.zeros([conv1_features], dtype=tf.float32))
conv2_weight=tf.Variable(tf.truncated_normal([4, 4, conv1_features, conv2_features], stddev=0.1, dtype=tf.float32))
conv2_bias=tf.Variable(tf.zeros([conv2_features], dtype=tf.float32))

In [273]:
# step 6: declare weight and bias for fully connected network 
resulting_width=image_width//(max_pool_size1* max_pool_size2)
resulting_height=image_height//(max_pool_size1*max_pool_size2)
full1_input_size=1*resulting_width*conv2_features
full1_weight=tf.Variable(tf.truncated_normal([full1_input_size, fully_connected_size1], stddev=0.1, dtype=tf.float32))
full1_bias=tf.Variable(tf.truncated_normal([fully_connected_size1], stddev=0.1, dtype=tf.float32))
full2_weight=tf.Variable(tf.truncated_normal([fully_connected_size1, target_size], stddev=0.1, dtype=tf.float32))
full2_bias=tf.Variable(tf.truncated_normal([1], stddev=0.1, dtype=tf.float32))

In [274]:
#step 7: declaring our entire CNN framework. (CNN + fully connected layers model)
def my_conv_net(input_data):
    # First Conv-ReLU-MaxPool Layer
    conv1=tf.nn.conv2d(input_data, conv1_weight, strides=[1,1,1,1], padding='SAME')
    relu1=tf.nn.relu(tf.nn.bias_add(conv1, conv1_bias))
    max_pool1=tf.nn.max_pool(relu1, ksize=[1, max_pool_size1, max_pool_size1, 1], strides=[1, max_pool_size1, max_pool_size1, 1], padding='SAME')
    
    # second Conv_ReLU_Maxpool layer
    conv2=tf.nn.conv2d(max_pool1, conv2_weight, strides=[1,1,1,1], padding='SAME')
    relu2=tf.nn.relu(tf.nn.bias_add(conv2, conv2_bias))
    max_pool2=tf.nn.max_pool(relu2, ksize=[1, max_pool_size2, max_pool_size2, 1], strides=[1, max_pool_size2, max_pool_size2, 1], padding='SAME')
    print(max_pool2.shape)
    # Transform output from CNN layer to Fully connceted layer
    
    final_conv_shape=max_pool2.get_shape().as_list()
    final_shape=final_conv_shape[1]*final_conv_shape[2]*final_conv_shape[3]
    print(final_conv_shape[0])
    flat_output=tf.reshape(max_pool2, [fully_connected_size1, final_shape])

    # First fully connected Layer
    fully_connected1=tf.nn.relu(tf.add(tf.matmul(flat_output, full1_weight), full1_bias))
    final_model_output=tf.add(tf.matmul(fully_connected1, full2_weight), full2_bias)
    
    return (final_model_output)
    

In [275]:
# step 8: declare model with train and test data. apply loss and accuracy function
model_output=my_conv_net(x_input)
print(model_output.shape)
test_model_output=my_conv_net(eval_input)
print(test_model_output.shape)
print(y_target.shape)
print(x_input.shape)
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_target, logits=model_output))

# # accuracy function
prediction=tf.nn.softmax(model_output)
test_prediction=tf.nn.softmax(test_model_output)

def get_accuracy(logits, targets):
    batch_predictions=np.argmax(logits, axis=1)
    num_correct=np.sum(np.equal(batch_predictions, targets))
    return (100. * num_correct/batch_predictions.shape[0])


(100, 1, 7, 50)
100
(100, 1)
(100, 1, 7, 50)
100
(100, 1)
(100, 1)
(100, 1, 28, 28)


In [276]:
# step 9: define optimizer function and initilize model variables
my_optimizer=tf.train.MomentumOptimizer(learning_rate, 0.9)
train_step=my_optimizer.minimize(loss)

# initialize variables
init=tf.initialize_all_variables()
sess.run(init)

In [280]:
#step 10: starting Training model
train_loss=[]
train_acc=[]
test_acc=[]
for i in range(generations):
    rand_index = np.random.choice(len(train_xdata), size=(batch_size))
    rand_x=train_xdata[rand_index]
    rand_x=np.expand_dims(rand_x,1) 
    rand_y=train_labels[rand_index]
    rand_y=np.expand_dims(rand_y,1)
    train_dict= {x_input:rand_x, y_target: rand_y}
    #print(rand_x.shape)
    #print(rand_y.shape)
    sess.run(train_step,  feed_dict=train_dict)
    temp_train_loss, temp_train_preds=sess.run([loss, prediction], feed_dict=train_dict)
    temp_train_acc = get_accuracy(temp_train_preds, rand_y)
    
    if (i+1)% eval_every==0:
        eval_index=np.random.choice(len(test_xdata), size=(evaluation_size))
       # print(len(eval_index))
        eval_x=test_xdata[eval_index]
        eval_x=np.expand_dims(eval_x, 1)
        eval_y=test_labels[eval_index]
        eval_y=np.expand_dims(eval_index, 1)
        test_dict={eval_input:eval_x, eval_target:eval_y}
        test_preds=sess.run(test_prediction, feed_dict=test_dict)
        temp_test_acc=get_accuracy(test_preds, eval_y)
        
        # record and print results
        train_loss.append(temp_train_loss)
        test_acc.append(temp_train_acc)
        test_acc.append(temp_test_acc)
        acc_and_loss = [(i+1), temp_train_loss, temp_train_acc, temp_test_acc]
        acc_and_loss = [np.round(x,2) for x in acc_and_loss]
        print('Generation # {}. Train loss: {:.2f}. Train Acc (Test Acc):{:.2f}({:.2f})'.format(*acc_and_loss))

Generation # 5. Train loss: nan. Train Acc (Test Acc):600.00(0.00)
Generation # 10. Train loss: nan. Train Acc (Test Acc):1100.00(0.00)
Generation # 15. Train loss: nan. Train Acc (Test Acc):500.00(0.00)
Generation # 20. Train loss: nan. Train Acc (Test Acc):800.00(0.00)
Generation # 25. Train loss: nan. Train Acc (Test Acc):1000.00(0.00)
Generation # 30. Train loss: nan. Train Acc (Test Acc):1100.00(0.00)
Generation # 35. Train loss: nan. Train Acc (Test Acc):1100.00(0.00)
Generation # 40. Train loss: nan. Train Acc (Test Acc):1900.00(0.00)
Generation # 45. Train loss: nan. Train Acc (Test Acc):800.00(0.00)
Generation # 50. Train loss: nan. Train Acc (Test Acc):900.00(0.00)
Generation # 55. Train loss: nan. Train Acc (Test Acc):300.00(0.00)
Generation # 60. Train loss: nan. Train Acc (Test Acc):800.00(0.00)
Generation # 65. Train loss: nan. Train Acc (Test Acc):700.00(0.00)
Generation # 70. Train loss: nan. Train Acc (Test Acc):600.00(0.00)
Generation # 75. Train loss: nan. Train Acc 